# Import Libraries and Prepare Returns Data

In [93]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
from sklearn import linear_model
import warnings
warnings.filterwarnings("ignore")

In [96]:
#Load Tactical Index Returns
tactical_index = pd.read_excel('TacticalIndexReturns_20210212.xlsx', skiprows = 1, index_col = 'Date')
tactical_index = tactical_index.iloc[:,[0,1]].replace('*', np.nan) #Replace values null values with nan
tactica_index_index = tactical_index.index
#read in SP500 and FF3 factors data
sp = yf.download('^SP500TR')
sp.columns = [f"SP500 {x}" for x in sp.columns]

ff = pd.read_csv('F-F_Research_Data_Factors.csv',index_col = 'Date',
                 skiprows = 3, parse_dates = True, skipfooter = 99)
ff = (ff / 100) #Divide by 100 to convert from % to decimal
ff_index = ff.index
#read in tactical data (RORO & TCUS)
atacx = yf.download('ATACX')
atacx.columns = [f"ATACX {x}" for x in atacx.columns]
first_atacx = atacx.index[0]

#load and format TCUS Returns
tcus = pd.read_excel('TCUS Returns.xlsx', index_col = 'Month')
tcus.columns = ['TCUS']
tcus_index = tcus.index

#Join DataFrames together on selected data 
df = pd.concat([tactical_index,ff, sp, atacx, tcus], axis = 1).fillna(method = 'ffill') #.fillna() to smooth end of month

df = df.loc[ff_index,['All','Filtered','ATACX Close', 'TCUS','Mkt-RF','SMB','HML','RF','SP500 Close']] # ff_index is the longest index of end of month returns
df['ATACX Returns'] = df['ATACX Close'].pct_change()
df['SP500 Returns'] = df['SP500 Close'].pct_change()
df = df.drop(['ATACX Close', 'SP500 Close'], axis = 1)
df = df.iloc[1:,:]
df.tail(10)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,All,Filtered,TCUS,Mkt-RF,SMB,HML,RF,ATACX Returns,SP500 Returns
Date,,,,,,,,,
2020-08-31,0.045176,0.038666,0.057602,0.0763,-0.0025,-0.0294,0.0001,-0.002515,0.071880
2020-09-30,-0.025827,-0.021815,-0.029419,-0.0363,0.0006,-0.0251,0.0001,-0.059664,-0.037997
2020-10-31,-0.025827,-0.021815,-0.036605,-0.0210,0.0444,0.0403,0.0001,-0.004915,-0.026593
2020-11-30,-0.025827,-0.021815,0.178086,0.1247,0.0548,0.0211,0.0001,0.048271,0.109464
2020-12-31,-0.025827,-0.021815,0.097485,0.0463,0.0481,-0.0136,0.0001,-0.017134,0.038449
2021-01-31,-0.025827,-0.021815,0.041290,-0.0003,0.0719,0.0285,0.0000,0.046197,-0.010096
2021-02-28,-0.025827,-0.021815,0.083357,0.0278,0.0211,0.0708,0.0000,-0.020412,0.027575
2021-03-31,-0.025827,-0.021815,0.033834,0.0308,-0.0248,0.0740,0.0000,-0.041676,0.043796
2021-04-30,-0.025827,-0.021815,0.039474,0.0493,-0.0309,-0.0074,0.0000,0.018416,0.053369


Create dummy variables for timing analysis -- take value of returns when 

In [97]:
df['SP500 Gamma * Dummy'] = np.where(df['SP500 Returns'] > 0,df['SP500 Returns'],0)
df['Mkt-RF Gamma * Dummy'] = np.where(df['Mkt-RF'] > 0,df['Mkt-RF'],0)

# Regression Order and Intuition 

Funds under consideration:
* Total tactical index ('All') - an index compiled by Good Harbor Financial of the returns of all funds that are considered 'tactical' by Morningstar.
* Filtered tactical index ('Filtered') - The aforementioned index, filtered by $\beta$ to the market.
* ATACX - A mutual fund that rotates around small-caps, large-caps, or emerging markets (risk-on), and Treasuries (risk-off) based on Utilities and Treasuries as risk triggers. (ATAC Funds Website)
* TCUS - seeks to "outperform the S&P 500 Index by aligning capital with the US equity market during sustained rallies and positioning defensively in weak equity market conditions." (TCUS Fund Overview)



Order is as follows:<br>
1 - Y: Total tactical index X: SP500 Index and SP500 Timing Var<br>
2 - Y: Total tactical index X: Mkt-RF Index and Mkt-RF Timing Var<br>
3 - Y: Total tactical index X: Mkt-RF Index and Mkt-RF Timing Var, FF3 SMB, HML<br>

4 - Y: Filtered tactical index X: SP500 Index and SP500 Timing Var<br>
5 - Y: Filtered tactical index X: Mkt-RF Index and Mkt-RF Timing Var<br>
6 - Y: Filtered tactical index X: Mkt-RF Index and Mkt-RF Timing Var, FF3 SMB, HML<br>

7 - Y: ATACX X: SP500 Index and SP500 Timing Var<br>
8 - Y: ATACX X: Mkt-RF Index and Mkt-RF Timing Var<br>
9 - Y: ATACX X: Mkt-RF Index and Mkt-RF Timing Var, FF3 SMB, HML<br>

10 - Y: TCUS X: SP500 Index and SP500 Timing Var<br>
11 - Y: TCUS X: Mkt-RF Index and Mkt-RF Timing Var<br>
12 - Y: TCUS X: Mkt-RF Index and Mkt-RF Timing Var, FF3 SMB, HML<br>

Using the methodology proposed by Henriksson and Merton (1981), the regressions below are of the format:

$$
R_{fund} = \alpha  +  \beta(r_{mkt} - rf)  +  \gamma((r_{mkt} - rf)*D(r_{mkt} > 0))
$$
Where: <br> 
D = 1, $r_{mkt} > 0$ <br>
D = 0, $r_{mkt} < 0$

With this multiple regression analysis:<br>
$\alpha$ suggests stock-picking ability<br> 
$\beta$ refers to market exposure (too high makes you a closet indexer)<br>
$\gamma$ suggests market timing ability

# Regression Results and Discussion

In [98]:
#1
tactical_index_data = df.loc[tactica_index_index[1:]] #Selecting on this index prevents unnecessary .ffill()
Y = tactical_index_data['All']
X = tactical_index_data.loc[:,['SP500 Returns','SP500 Gamma * Dummy']]
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    All   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.755
Method:                 Least Squares   F-statistic:                     382.8
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           2.97e-76
Time:                        23:28:01   Log-Likelihood:                 663.18
No. Observations:                 249   AIC:                            -1320.
Df Residuals:                     246   BIC:                            -1310.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.0032      0.002      1.865      0.063      -0.000       0.007
SP500 Returns           0.7035      0.044     15.966      0.000       0.617       0.790
SP500 Gamma * Dummy    -0.0382      0.077     -0.496      0.620      -0.190       0.114
==============================================================================
Omnibus:                      262.508   Durbin-Watson:                   2.276
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15923.454
Skew:                           4.042   Prob(JB):                         0.00
Kurtosis:                      41.333   Cond. No.                         79.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [99]:
#2
Y = tactical_index_data['All']
X = tactical_index_data.loc[:,['Mkt-RF','Mkt-RF Gamma * Dummy']]
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    All   R-squared:                       0.818
Model:                            OLS   Adj. R-squared:                  0.817
Method:                 Least Squares   F-statistic:                     553.2
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           9.06e-92
Time:                        23:28:02   Log-Likelihood:                 699.34
No. Observations:                 249   AIC:                            -1393.
Df Residuals:                     246   BIC:                            -1382.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0027      0.001      1.814      0.071      -0.000       0.006
Mkt-RF                   0.6730      0.036     18.529      0.000       0.601       0.744
Mkt-RF Gamma * Dummy     0.0266      0.065      0.406      0.685      -0.102       0.155
==============================================================================
Omnibus:                      242.710   Durbin-Watson:                   2.122
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10531.422
Skew:                           3.675   Prob(JB):                         0.00
Kurtosis:                      34.001   Cond. No.                         77.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [100]:
#3
Y = tactical_index_data['All']
X = tactical_index_data.loc[:,['Mkt-RF','Mkt-RF Gamma * Dummy','SMB','HML']]
X = sm.add_constant(X)

model = sm.OLS(Y,X, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    All   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     384.1
Date:                Wed, 08 Sep 2021   Prob (F-statistic):          5.29e-104
Time:                        23:28:02   Log-Likelihood:                 734.59
No. Observations:                 249   AIC:                            -1459.
Df Residuals:                     244   BIC:                            -1442.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0020      0.001      1.520      0.130      -0.001       0.005
Mkt-RF                   0.6154      0.032     19.050      0.000       0.552       0.679
Mkt-RF Gamma * Dummy     0.0521      0.057      0.913      0.362      -0.060       0.165
SMB                      0.2359      0.027      8.770      0.000       0.183       0.289
HML                      0.0890      0.025      3.494      0.001       0.039       0.139
==============================================================================
Omnibus:                      133.356   Durbin-Watson:                   2.160
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2076.225
Skew:                           1.719   Prob(JB):                         0.00
Kurtosis:                      16.722   Cond. No.                         78.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Tactical Index Discussion

Regression analysis of the entire tactical index highlights some key points:
* The index as a whole exhibits high beta to both the S&P500 and the total market (.6-.7). This indicates that a large portion of the index's returns are attributable to market $\beta$ /closet indexing by individual managers. 
    * These coefficients are statistically significant for each regression analysis.
* When using both the S&P 500 and the total market index, the analysis does not suggest any significant market timing ability. None of the three $\gamma$ coefficients are significant at any significance level.
    * This suggests that market timing by managers in the total tactical index does not improve their returns.
* Much of the tactical index returns appear to be explained by exposure to size and value factors, rather than market timing.

In [101]:
#4
Y = tactical_index_data['Filtered']
X = tactical_index_data.loc[:,['SP500 Returns','SP500 Gamma * Dummy']]
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Filtered   R-squared:                       0.552
Model:                            OLS   Adj. R-squared:                  0.548
Method:                 Least Squares   F-statistic:                     151.5
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           1.31e-43
Time:                        23:28:03   Log-Likelihood:                 614.65
No. Observations:                 249   AIC:                            -1223.
Df Residuals:                     246   BIC:                            -1213.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.0037      0.002      1.791      0.074      -0.000       0.008
SP500 Returns           0.5348      0.054      9.989      0.000       0.429       0.640
SP500 Gamma * Dummy    -0.0228      0.094     -0.243      0.808      -0.207       0.162
==============================================================================
Omnibus:                      291.465   Durbin-Watson:                   2.184
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            25503.500
Skew:                           4.687   Prob(JB):                         0.00
Kurtosis:                      51.686   Cond. No.                         79.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [102]:
#5
Y = tactical_index_data['Filtered']
X = tactical_index_data.loc[:,['Mkt-RF','Mkt-RF Gamma * Dummy']]
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Filtered   R-squared:                       0.630
Model:                            OLS   Adj. R-squared:                  0.627
Method:                 Least Squares   F-statistic:                     209.8
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           6.86e-54
Time:                        23:28:03   Log-Likelihood:                 638.61
No. Observations:                 249   AIC:                            -1271.
Df Residuals:                     246   BIC:                            -1261.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0032      0.002      1.680      0.094      -0.001       0.007
Mkt-RF                   0.5243      0.046     11.312      0.000       0.433       0.616
Mkt-RF Gamma * Dummy     0.0307      0.083      0.367      0.714      -0.134       0.195
==============================================================================
Omnibus:                      285.097   Durbin-Watson:                   2.100
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22096.669
Skew:                           4.559   Prob(JB):                         0.00
Kurtosis:                      48.240   Cond. No.                         77.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [103]:
#6
Y = tactical_index_data['Filtered']
X = tactical_index_data.loc[:,['Mkt-RF','Mkt-RF Gamma * Dummy','SMB','HML']]
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Filtered   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.709
Method:                 Least Squares   F-statistic:                     151.8
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           5.47e-65
Time:                        23:28:04   Log-Likelihood:                 670.27
No. Observations:                 249   AIC:                            -1331.
Df Residuals:                     244   BIC:                            -1313.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0025      0.002      1.465      0.144      -0.001       0.006
Mkt-RF                   0.4563      0.042     10.909      0.000       0.374       0.539
Mkt-RF Gamma * Dummy     0.0551      0.074      0.746      0.456      -0.090       0.201
SMB                      0.2916      0.035      8.371      0.000       0.223       0.360
HML                      0.0326      0.033      0.989      0.324      -0.032       0.098
==============================================================================
Omnibus:                      168.896   Durbin-Watson:                   2.113
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3752.009
Skew:                           2.271   Prob(JB):                         0.00
Kurtosis:                      21.466   Cond. No.                         78.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Filtered Tactical Index Discussion

Regression analysis again provides similar findings when filtering tactical managers based on their market $\beta$.
* Analysis of the filtered index yields similar results to the entire index. Analysis shows lower market $\beta$ to the market. This to be expected as managers are specifically selected based on low market $\beta$
* Despite this lower market $\beta$, our analysis does not suggest any significant market timing abilities by these managers.
    * Instead this subset of managers appears to be adding much of their value from exposure to the FF3 size factor. 

In [104]:
#CAPM Model
Y = df.loc[first_roro:,'ATACX Returns']
X = df.loc[first_roro:,['SP500 Returns']]
pd.concat([Y, X], axis = 1)
X = sm.add_constant(X)
model = sm.OLS(Y,X, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          ATACX Returns   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.131
Method:                 Least Squares   F-statistic:                     16.57
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           9.28e-05
Time:                        23:28:04   Log-Likelihood:                 178.34
No. Observations:                 104   AIC:                            -352.7
Df Residuals:                     102   BIC:                            -347.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0011      0.005      0.238      0.812      -0.008       0.010
SP500 Returns     0.4595      0.113      4.070      0.000       0.236       0.683
==============================================================================
Omnibus:                        1.229   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.541   Jarque-Bera (JB):                1.246
Skew:                           0.252   Prob(JB):                        0.536
Kurtosis:                       2.817   Cond. No.                         26.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [105]:
#7
Y = df.loc[first_roro:,'ATACX Returns']
X = df.loc[first_roro:,['SP500 Returns', 'SP500 Gamma * Dummy']]
pd.concat([Y, X], axis = 1)
X = sm.add_constant(X)
model = sm.OLS(Y,X, missing = 'drop')
results = model.fit()
results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          ATACX Returns   R-squared:                       0.186
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                     11.51
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           3.14e-05
Time:                        23:28:05   Log-Likelihood:                 181.19
No. Observations:                 104   AIC:                            -356.4
Df Residuals:                     101   BIC:                            -348.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.0105      0.007     -1.597      0.113      -0.024       0.003
SP500 Returns           0.0196      0.215      0.091      0.928      -0.407       0.446
SP500 Gamma * Dummy     0.7896      0.331      2.385      0.019       0.133       1.446
==============================================================================
Omnibus:                        0.146   Durbin-Watson:                   2.159
Prob(Omnibus):                  0.930   Jarque-Bera (JB):                0.260
Skew:                          -0.081   Prob(JB):                        0.878
Kurtosis:                       2.817   Cond. No.                         90.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [106]:
#8
Y = df['ATACX Returns']
X = df.loc[:,['Mkt-RF','Mkt-RF Gamma * Dummy']]
X = sm.add_constant(X)
model = sm.OLS(Y,X, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          ATACX Returns   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     12.33
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           1.62e-05
Time:                        23:28:05   Log-Likelihood:                 181.87
No. Observations:                 104   AIC:                            -357.7
Df Residuals:                     101   BIC:                            -349.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0095      0.006     -1.494      0.138      -0.022       0.003
Mkt-RF                   0.0654      0.199      0.328      0.743      -0.330       0.461
Mkt-RF Gamma * Dummy     0.7065      0.305      2.315      0.023       0.101       1.312
==============================================================================
Omnibus:                        0.155   Durbin-Watson:                   2.181
Prob(Omnibus):                  0.925   Jarque-Bera (JB):                0.317
Skew:                          -0.062   Prob(JB):                        0.854
Kurtosis:                       2.759   Cond. No.                         84.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [107]:
#9
Y = df['ATACX Returns']
X = df.loc[:,['Mkt-RF','Mkt-RF Gamma * Dummy', 'SMB','HML']]
X = sm.add_constant(X)

model = sm.OLS(Y,X, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          ATACX Returns   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     7.295
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           3.43e-05
Time:                        23:28:05   Log-Likelihood:                 183.95
No. Observations:                 104   AIC:                            -357.9
Df Residuals:                      99   BIC:                            -344.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0084      0.006     -1.327      0.188      -0.021       0.004
Mkt-RF                   0.1122      0.210      0.533      0.595      -0.305       0.529
Mkt-RF Gamma * Dummy     0.5960      0.312      1.909      0.059      -0.024       1.216
SMB                      0.2311      0.174      1.328      0.187      -0.114       0.576
HML                     -0.2357      0.143     -1.651      0.102      -0.519       0.047
==============================================================================
Omnibus:                        0.821   Durbin-Watson:                   2.229
Prob(Omnibus):                  0.663   Jarque-Bera (JB):                0.846
Skew:                          -0.021   Prob(JB):                        0.655
Kurtosis:                       2.560   Cond. No.                         88.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## ATACX Strategy Discussion

Results from the ATACX analysis are a bit perplexing. A simple CAPM model shows an SP500 $\beta$ of .46. This model has a much lower $R^2$ value than any of the tactical index models shown prior. Adding the market timing factor provides little improvement to the overall model fit. Alternatively, it suggests that there is market-timing ability in the strategy and this coefficient is significant. At the same time, $\beta_{mkt}$ drops by roughly .3 and the coefficient loses significance when using both the S&P 500 and the FF3 total market as the index. 

In [108]:
# CAPM Model
Y = df['TCUS']
X = df.loc[:,['SP500 Returns']]
X = sm.add_constant(X)

model = sm.OLS(Y,X, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TCUS   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.343
Method:                 Least Squares   F-statistic:                     190.5
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           4.06e-35
Time:                        23:28:06   Log-Likelihood:                 756.35
No. Observations:                 364   AIC:                            -1509.
Df Residuals:                     362   BIC:                            -1501.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0078      0.002      4.750      0.000       0.005       0.011
SP500 Returns     0.5252      0.038     13.804      0.000       0.450       0.600
==============================================================================
Omnibus:                       28.803   Durbin-Watson:                   1.912
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               39.692
Skew:                           0.582   Prob(JB):                     2.40e-09
Kurtosis:                       4.124   Cond. No.                         23.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [109]:
#10
Y = df['TCUS']
X = df.loc[:,['SP500 Returns','SP500 Gamma * Dummy']]
X = sm.add_constant(X)

model = sm.OLS(Y,X, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TCUS   R-squared:                       0.392
Model:                            OLS   Adj. R-squared:                  0.389
Method:                 Least Squares   F-statistic:                     116.5
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           8.94e-40
Time:                        23:28:06   Log-Likelihood:                 770.04
No. Observations:                 364   AIC:                            -1534.
Df Residuals:                     361   BIC:                            -1522.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.0024      0.002     -0.964      0.336      -0.007       0.002
SP500 Returns           0.2162      0.069      3.143      0.002       0.081       0.351
SP500 Gamma * Dummy     0.6083      0.115      5.311      0.000       0.383       0.833
==============================================================================
Omnibus:                        8.899   Durbin-Watson:                   2.017
Prob(Omnibus):                  0.012   Jarque-Bera (JB):               14.820
Skew:                           0.087   Prob(JB):                     0.000605
Kurtosis:                       3.973   Cond. No.                         84.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [110]:
#11
Y = df['TCUS']
X = df.loc[:,['Mkt-RF','Mkt-RF Gamma * Dummy']]
X = sm.add_constant(X)

model = sm.OLS(Y,X, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TCUS   R-squared:                       0.436
Model:                            OLS   Adj. R-squared:                  0.433
Method:                 Least Squares   F-statistic:                     139.5
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           1.31e-45
Time:                        23:28:06   Log-Likelihood:                 783.58
No. Observations:                 364   AIC:                            -1561.
Df Residuals:                     361   BIC:                            -1549.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0025      0.002     -1.048      0.295      -0.007       0.002
Mkt-RF                   0.2205      0.062      3.541      0.000       0.098       0.343
Mkt-RF Gamma * Dummy     0.6442      0.107      6.009      0.000       0.433       0.855
==============================================================================
Omnibus:                        7.233   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.027   Jarque-Bera (JB):                9.769
Skew:                          -0.149   Prob(JB):                      0.00756
Kurtosis:                       3.745   Cond. No.                         81.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [111]:
#12
Y = df['TCUS']
X = df.loc[:,['Mkt-RF','Mkt-RF Gamma * Dummy', 'SMB', 'HML']]
X = sm.add_constant(X)

model = sm.OLS(Y,X, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TCUS   R-squared:                       0.469
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     79.28
Date:                Wed, 08 Sep 2021   Prob (F-statistic):           3.79e-48
Time:                        23:28:07   Log-Likelihood:                 794.60
No. Observations:                 364   AIC:                            -1579.
Df Residuals:                     359   BIC:                            -1560.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0037      0.002     -1.570      0.117      -0.008       0.001
Mkt-RF                   0.1641      0.062      2.658      0.008       0.043       0.286
Mkt-RF Gamma * Dummy     0.6965      0.105      6.619      0.000       0.490       0.903
SMB                      0.2186      0.048      4.586      0.000       0.125       0.312
HML                      0.0983      0.047      2.080      0.038       0.005       0.191
==============================================================================
Omnibus:                        9.942   Durbin-Watson:                   2.048
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               12.391
Skew:                          -0.262   Prob(JB):                      0.00204
Kurtosis:                       3.737   Cond. No.                         82.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## TCUS Discussion

Regression analysis of Tactical Core US provides some interesting findings.
* When using the S&P 500 as the benchmark, this analysis suggests positive significant market timing ability coupled with low $\beta_{mkt}$.
* All three regressions yield significant coefficients to the market timing factor which suggests that the strategy holds market-timing capability.
* These results suggest that TCUS exhibits some positive market timing ability. Additionally, it's returns can be attributed to exposure to the FF3 size and value factors as well. 

# Conclusion

In conclusion, our analysis yields a few interesting results:
1. "Tactical" managers as an asset-class can best be classified as low-beta closet indexers. This can be seen by the high $R^2$ value, but $\beta < 1$. 
2. As an asset-class, tactical managers do not provide investors with any significant market-timing capabilities.
3. ATACX shows low $\beta_{mkt}$, but high market-timing capabilities. 
4. TCUS shows both $\beta_{mkt}$, but high market-timing capabilities as well, indicating that both TCUS and ATACX can attribute their returns to upside market capture.

Further research could focus on how this relationship has changed since quantitative easing has led to a relative decline in market volatility. 